In [1]:
import utils
import numpy as np
import math
import matplotlib.pyplot as plt

from scipy.signal import medfilt
from plotly import tools
import chart_studio.plotly as py
import plotly.graph_objs as go
from pose_parser import parse
import pose


NameError: name 'Joint' is not defined

In [10]:
ps = parse("dataset/bicep/front_bicep.npy")
print(ps[0])

Data shape:  (319, 18, 3)
Mean torso:  152.61040275513415
nose <1.5470439481037097, 0.4133512451390067, 0.843448>
neck <1.5470373954704557, 0.6376518130034632, 0.8602>
rshoulder <1.3676066390761066, 0.6436717171739152, 0.810499>
relbow <1.3227735223522217, 0.944863504694138, 0.821051>
rwrist <1.3033384121208504, 1.2523392675049494, 0.866991>
lshoulder <1.7197254922464364, 0.6374467155826129, 0.831475>
lelbow <1.7709539790262279, 0.9383632925061677, 0.823595>
lwrist <1.8096472783911122, 1.2332186836697703, 0.799488>
rhip <1.5468539217393438, 1.200927306994046, 0.62752>
rknee <1.425263259078074, 1.1947219633025057, 0.607082>
rankle <1.4126166768978494, 1.630216521996763, 0.763935>
lhip <1.4125052821325312, 2.078567347135373, 0.83464>
lknee <1.662068872244469, 1.1946629896032197, 0.651334>
lankle <1.662121293310501, 1.6239653098724447, 0.752946>
reye <1.662101635410739, 2.0722571613117684, 0.799184>
leye <1.5086127540689853, 0.38134818432645873, 0.937325>
rear <1.5789421667841936, 0.38119

AttributeError: type object 'Joint' has no attribute 'vectorFromJoints'